In [ ]:
from konlpy.tag import *
import pandas as pd
import numpy as np
import re

from gensim.models import Word2Vec

twitter = Okt()
kkma = Kkma()

# 아이디와 본문 분리
def clean_text(df):
    if(len(df['x'].split(":")) == 1):
        return df['x'].split(":")[0]
    return str(df['x'].split(":")[1])

# 한글 만 뽑아내기
def clean_text2(df):
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub("", df['x'])
    return str(result)

def tokenazing(texts):
    temp_list = []
    for i in texts:
        temp = twitter.nouns(i)
        temp_list.append(temp)
    return temp_list


data = pd.read_csv('data/tw_Cr.csv')

data['x'] = data.apply(clean_text2, axis=1)

sentences = tokenazing(data['x'])

CB_Model = Word2Vec(sentences, sg=0, window=5, size=10, min_count=2) 
result = dict(zip(CB_Model.wv.index2word, CB_Model.wv.vectors))
word_vectors = CB_Model.wv.vectors

from sklearn.cluster import KMeans
kmeans_clustering = KMeans(n_clusters= len(set(data['Keyword'])))
idx = kmeans_clustering.fit_predict(word_vectors)

from gensim.models import FastText

FT_Model = FastText(sentences, sg=1, window=5, size=10, min_count=1, min_n=1, max_n=2)

# FT_Model.wv.most_similar('자유한국당')

for i in data['Keyword'].unique():
    print(i)
    print(FT_Model.wv.most_similar(i))

word_vectors2 = FT_Model.wv.vectors

type(data['Keyword'])

data['labels'] = data['Keyword'].astype('category')

kmeans_clustering2 = KMeans(n_clusters= len(set(data['Keyword'])))
idx2 = kmeans_clustering2.fit_predict(word_vectors2)

most_similar = []
for i in data['Keyword'].unique():
    most_similar.append(FT_Model.wv.most_similar(i, topn=15))

result_data = {}
result_data['feature'] = FT_Model.wv.index2word
result_data['label'] = idx2
result_data = pd.DataFrame(result_data)
result_data

### 가장 유사한 단어들 뽑아내기
result_data2 = {}
for i in range(0,13):
    temp = []
    for j in range(0,15):
        temp.append(most_similar[i][j][0])
    result_data2[i] = temp

#### 군집별 단어 뽑아내기
result_data3 = {}
for i in range(0,13):
    result_data3[i] = result_data.feature[result_data.label == i].values.reshape(-1)


### 비교 테이블 작성
check_box = []
for i in range(0,13):
    line_point = np.zeros(13)
    for j in range(0,15):
        for k in range(0,13):
            if(result_data2[i][j] in result_data3[k]):
                line_point[k] = line_point[k]+1
    check_box.append(line_point)

check_box = pd.DataFrame(check_box)

check_box.columns = data['Keyword'].unique()

# 행 kmeans 라벨 /  열 키워드 라벨
check_box = check_box.astype('int') # 엉망 진창 와진창.


check_box

In [119]:
check_box

,지방선거,서울시장,박원순,김문수,안철수,자유한국당,더불어민주당,바른미래당,미투,문재인,보수,진보,남북정상회담
0,0,6,0,0,0,0,3,0,1,5,0,0,0
1,1,4,0,0,0,0,0,8,2,0,0,0,0
2,0,5,0,0,0,1,0,1,1,0,0,6,1
3,0,3,0,0,0,0,0,4,0,0,0,2,6
4,0,11,0,0,2,0,1,0,0,0,1,0,0
5,0,3,8,0,0,0,4,0,0,0,0,0,0
6,2,5,0,0,0,0,1,1,0,0,0,2,4
7,0,1,0,0,0,0,0,14,0,0,0,0,0
8,5,0,7,0,0,0,0,0,0,0,1,0,2
9,0,4,0,0,0,0,5,0,5,1,0,0,0


In [ ]:
# 10 -> 박원순
# 12 -> 미투
# 4 -> 서울시장
# 7 -> 바른미래당
# 11 -> 안철수